## problema caxeiro viajante aluguel de carros 



# Instance creator

In [1]:
import random 

def generate_symmetric_matrix(size):
    matrix = [[0 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(i, size):
            if(i!=j):
                value = random.randint(1, 100)  # Random integer between 1 and 100
                matrix[i][j] = value
                matrix[j][i] = value  
    return matrix

def generate_cars_instance_file(citys, cars, file_name):
    with open(file_name, 'w') as file:
        file.write(f"NAME : Test{citys}n\n")
        file.write("TYPE : CaRS\n")
        file.write("COMMENT : Instances for the CaRS Problem (Asconavieta & Goldbarg)\n")
        file.write(f"DIMENSION : {citys}\n")
        file.write(f"CARS_NUMBER : {cars}\n")
        file.write("EDGE_WEIGHT_TYPE : EXPLICIT\n")
        file.write("EDGE_WEIGHT_FORMAT : FULL_MATRIX\n")

        file.write("EDGE_WEIGHT_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")


        file.write("RETURN_RATE_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")

        file.write("EOF\n")

# Example usage:
generate_cars_instance_file(4,2, "teste.car")


# Instance reader

In [2]:
class CaRSInstance:
    def __init__(self):
        self.name = ""
        self.type = ""
        self.comment = ""
        self.dimension = 0
        self.cars_number = 0
        self.edge_weight_type = ""
        self.edge_weight_format = ""

class CaRSCar:
    def __init__(self, car_id):
        self.car_id = car_id
        self.distance_matrix = []
        self.return_rate_matrix = []

def read_cars_instance(file_path):
    cars_instance = CaRSInstance()
    cars = []  # List to store Car objects

    with open(file_path, 'r') as file:
        lines = file.readlines()

    section = None
    car_id = None  # To keep track of the current car
    edge_weight_section = False
    return_rate_section = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("EDGE_WEIGHT_SECTION"):
            edge_weight_section = True
            section = "EDGE_WEIGHT"
            continue
        elif line.startswith("RETURN_RATE_SECTION"):
            return_rate_section = True
            edge_weight_section = False
            section = "RETURN_RATE"
            continue
        elif line.startswith("EOF"):
            section = None  # Mark the end of the section
            continue

        if edge_weight_section:
            if line.isdigit():
                car_id = int(line)
                cars.append(CaRSCar(car_id))
            else:
                values = line.split()
                cars[-1].distance_matrix.append([int(value) for value in values])
        elif return_rate_section:
            if line.isdigit():
                car_id = int(line)
            else:
                values = line.split()
                cars[car_id].return_rate_matrix.append([int(value) for value in values])

    for line in lines:
        key_value = line.split(":")
        if len(key_value) == 2:
            key, value = key_value
            key = key.strip()
            value = value.strip()

            if key == "NAME":
                cars_instance.name = value
            elif key == "TYPE":
                cars_instance.type = value
            elif key == "COMMENT":
                cars_instance.comment = value
            elif key == "DIMENSION":
                cars_instance.dimension = int(value)
            elif key == "CARS_NUMBER":
                cars_instance.cars_number = int(value)
            elif key == "EDGE_WEIGHT_TYPE":
                cars_instance.edge_weight_type = value
            elif key == "EDGE_WEIGHT_FORMAT":
                cars_instance.edge_weight_format = value

    return cars_instance, cars

# Usage example
file_path = "teste.car"  
cars_instance, cars = read_cars_instance(file_path)

# Access the parsed data
print("Name:", cars_instance.name)
print("Type:", cars_instance.type)
print("Comment:", cars_instance.comment)
print("Dimension:", cars_instance.dimension)
print("Cars Number:", cars_instance.cars_number)
print("Edge Weight Type:", cars_instance.edge_weight_type)
print("Edge Weight Format:", cars_instance.edge_weight_format)


Name: Test4n
Type: CaRS
Comment: Instances for the CaRS Problem (Asconavieta & Goldbarg)
Dimension: 4
Cars Number: 2
Edge Weight Type: EXPLICIT
Edge Weight Format: FULL_MATRIX


In [3]:

# Access the distance matrices for each car
for car in cars:
    print(f"Car {car.car_id} Distance Matrix:")
    for row in car.distance_matrix:
        print(row)

# Access the return rate matrices for each car
for car in cars:
    print(f"Car {car.car_id} Return Rate Matrix:")
    for row in car.return_rate_matrix:
        print(row)

Car 0 Distance Matrix:
[0, 92, 83, 95]
[92, 0, 50, 73]
[83, 50, 0, 92]
[95, 73, 92, 0]
Car 1 Distance Matrix:
[0, 34, 70, 92]
[34, 0, 72, 36]
[70, 72, 0, 73]
[92, 36, 73, 0]
Car 0 Return Rate Matrix:
[0, 3, 47, 97]
[3, 0, 45, 9]
[47, 45, 0, 67]
[97, 9, 67, 0]
Car 1 Return Rate Matrix:
[0, 71, 54, 68]
[71, 0, 93, 8]
[54, 93, 0, 63]
[68, 8, 63, 0]


## Brute Force

In [4]:
import itertools
import time

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance

def backtrack_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance(full_route, car_assignments, cars)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = backtrack_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 0.00016689300537109375 segundos
Melhor rota: (0, 1, 3, 2, 0)
Melhor distância: 213
best_car_assignments: (1, 1, 1, 1)


## Brute Force 2

In [5]:
import itertools
import time

def calculate_total_distance(route, car_assignments, cars, best_distance):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]
        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
        if best_distance < total_distance:
            return total_distance

        current_location = next_location

    return total_distance

def branch_and_bound_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance(full_route, car_assignments, cars, best_distance)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = branch_and_bound_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 0.0001671314239501953 segundos
Melhor rota: (0, 1, 3, 2, 0)
Melhor distância: 213
best_car_assignments: (1, 1, 1, 1)


In [6]:
for citys in range(4,10):
    for cars_number in range(2,5):
        # Example usage:
        print(citys, "  ",cars_number )

        for i in range(0 ,3):
            generate_cars_instance_file(citys,cars_number, "teste.car")

            file_path = "teste.car" 
            cars_instance, cars = read_cars_instance(file_path)

            start_time = time.time()

            best_route, best_distance, best_car_assignments = branch_and_bound_tsp_with_cars(cars_instance, cars)

            end_time = time.time()
            print(i, "  ", end_time - start_time )


4    2
0    0.0002837181091308594
1    8.511543273925781e-05
2    9.012222290039062e-05
4    3
0    0.0003528594970703125
1    0.00036835670471191406
2    0.0003628730773925781
4    4
0    0.0009639263153076172
1    0.0012540817260742188
2    0.0016911029815673828
5    2
0    0.0012869834899902344
1    0.0009088516235351562
2    0.0006961822509765625
5    3
0    0.006475925445556641
1    0.006208181381225586
2    0.0050737857818603516
5    4
0    0.024984121322631836
1    0.020535707473754883
2    0.0265960693359375
6    2
0    0.007122039794921875


1    0.010009050369262695
2    0.0075299739837646484
6    3
0    0.1107330322265625
1    0.07928013801574707
2    0.0632929801940918
6    4
0    0.33014583587646484
1    0.33315110206604004
2    0.31369996070861816
7    2
0    0.06877613067626953
1    0.07521486282348633
2    0.08038091659545898
7    3
0    1.1590418815612793
1    1.109802007675171
